In [4]:
!pip install mesa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 13.9 MB/s eta 0:0

In [5]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import seaborn as sn

In [6]:
from google.colab import  files
uploaded = files.upload()

Saving mapa.txt to mapa.txt


In [2]:
class Cell():
  def __init__(self, x, y, wall):
    self.pos = (x, y)

    if wall[0] == '1': self.up = True
    else: self.up = False
    if wall[1] == '1': self.left = True
    else: self.left = False
    if wall[2] == '1': self.down = True
    else: self.down = False
    if wall[3] == '1': self.right = True
    else: self.right = False

    self.alert = 0 # 1- false alarm    2 - victim
    self.fire = 0 # 1 - smoke    2 - fire
    self.door = [] # pos of the connected cell with a door & if is open    [] if no door
    self.entrance = False # True if the current cell is a access point to the structure

In [21]:
class FiremanAgent(Agent):
    def __init__(self, unique_id, model, point=None):
      super().__init__(unique_id, model)
      self.point = point
      self.actionPoints = 4
      self.carryState = 1
      self.path = []

    def step(self):
      self.path = self.dijkstra(self.pos, self.point.pos)[0]
      while self.actionPoints >= 1 and self.pos != self.point.pos:
        self.actionPoints -= self.clearPath(self.pos, self.path[0])
        self.model.grid.move_agent(self, self.path[0])
        self.path.pop(0)
      if self.pos == self.point.pos:
          if self.point.alert == 2:
            print(f"Persona encontrada en {self.pos}")
            self.carryState = 2
            closestExit = None
            minDistance = 100
            for exit in self.model.outSide:
              distance = self.dijkstra(self.pos, exit.pos)[1]
              if distance < minDistance:
                minDistance = distance
                closestExit = exit
            for cell in self.model.interestPoints:
              if cell.pos == self.point.pos:
                self.model.interestPoints.remove(cell)
            self.path = self.dijkstra(self.pos, closestExit.pos)[0]
            self.point = closestExit
          elif self.model.cells[self.pos[0]][self.pos[1]] in self.model.outSide and self.carryState == 2:
            print(f"Vida salvada en {self.pos}")
            self.carryState = 1
            self.model.savedLifes += 1
            self.point = None
          elif self.point.alert == 1:
            print(f"Falsa alarma en {self.pos}")
            for cell in self.model.interestPoints:
              if cell.pos == self.point.pos:
                self.model.interestPoints.remove(cell)
            self.point = None
          elif self.point.fire == 2:
            self.point = None
      self.calculateActionPoints()

    def dijkstra(self, start, end):
      dijkstraMap = {}
      path = []
      for x in range(self.model.grid.height):
        for y in range(self.model.grid.width):
          dijkstraMap[(y, x)] = {"previousCell": None, "steps": None}
      if start in dijkstraMap and end in dijkstraMap and start != end:
        dijkstraMap[start]["steps"] = 0
        dijkstraMap[start]["previousCell"] = start
        queue = [start]
        while len(queue) > 0:
          for neighbor in self.model.grid.get_neighborhood(queue[0], moore=False):
            if 0 <= neighbor[0] < self.model.grid.width and 0 <= neighbor[1] < self.model.grid.height:
              if dijkstraMap[neighbor]["steps"] is None:
                dijkstraMap[neighbor]["steps"] = self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]
                dijkstraMap[neighbor]["previousCell"] = queue[0]
                queue.append(neighbor)
              elif (dijkstraMap[neighbor]["steps"] > self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"]):
                dijkstraMap[neighbor]["steps"] = (self.calculateSteps(queue[0], neighbor) + dijkstraMap[queue[0]]["steps"])
                dijkstraMap[neighbor]["previousCell"] = queue[0]
                queue.append(neighbor)
          queue.pop(0)
        cell = end
        while cell != start:
          path.insert(0, cell)
          cell = dijkstraMap[cell]["previousCell"]
        return path, dijkstraMap[end]["steps"]
      else:
        print("Start es igual a End")
        return [end], 0

    def calculateSteps(self, start, end):
          actionPointsCost = 0
          if 0 <= end[0] < len(self.model.cells) and 0 <= end[1] < len(self.model.cells[0]):
            if start[0] < end[0]:
              if (self.model.cells[end[0]][end[1]].up or self.model.cells[start[0]][start[1]].down) and end not in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 4.1
              elif (self.model.cells[end[0]][end[1]].up or self.model.cells[start[0]][start[1]].down) and end in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 1
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
              actionPointsCost = actionPointsCost + 1 * self.carryState
            elif start[0] > end[0]:
              if (self.model.cells[end[0]][end[1]].down or self.model.cells[start[0]][start[1]].up) and end not in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 4.1
              elif (self.model.cells[end[0]][end[1]].down or self.model.cells[start[0]][start[1]].up) and end in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 1
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
              actionPointsCost = actionPointsCost + 1 * self.carryState
            elif start[1] < end[1]:
              if (self.model.cells[end[0]][end[1]].left or self.model.cells[start[0]][start[1]].right) and end not in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 4.1
              elif (self.model.cells[end[0]][end[1]].left or self.model.cells[start[0]][start[1]].right) and end in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 1
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
              actionPointsCost = actionPointsCost + 1 * self.carryState
            elif start[1] > end[1]:
              if (self.model.cells[end[0]][end[1]].right or self.model.cells[start[0]][start[1]].left) and end not in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 4.1
              elif (self.model.cells[end[0]][end[1]].right or self.model.cells[start[0]][start[1]].left) and end in self.model.cells[start[0]][start[1]].door:
                actionPointsCost += 1
              if self.model.cells[end[0]][end[1]].fire == 2:
                actionPointsCost += 1
              actionPointsCost = actionPointsCost + 1 * self.carryState
          return actionPointsCost

    def calculateActionPoints(self):
      if self.actionPoints + 4 > 8:
        self.actionPoints = 8
      else:
        self.actionPoints += 4

    def removeWall(self, end):
      if self.pos[0] < end[0]:
        self.model.cells[end[0]][end[1]].up = False
        self.model.cells[self.pos[0]][self.pos[1]].down = False
        self.model.structural_Damage_Left -= 2
      elif self.pos[0] > end[0]:
        self.model.cells[end[0]][end[1]].down = False
        self.model.cells[self.pos[0]][self.pos[1]].up = False
        self.model.structural_Damage_Left -= 2
      elif self.pos[1] < end[1]:
        self.model.cells[end[0]][end[1]].left = False
        self.model.cells[self.pos[0]][self.pos[1]].right = False
        self.model.structural_Damage_Left -= 2
      elif self.pos[1] > end[1]:
        self.model.cells[end[0]][end[1]].right = False
        self.model.cells[self.pos[0]][self.pos[1]].left = False
        self.model.structural_Damage_Left -= 2

    def clearPath(self, start, end):
      actionPointsCost = 0
      if 0 <= end[0] < len(self.model.cells) and 0 <= end[1] < len(self.model.cells[0]):
        if start[0] < end[0]:
          if (self.model.cells[end[0]][end[1]].up or self.model.cells[start[0]][start[1]].down) and end not in self.model.cells[start[0]][start[1]].door:
            actionPointsCost += 4
            self.removeWall(end)
          elif (self.model.cells[end[0]][end[1]].up or self.model.cells[start[0]][start[1]].down) and end in self.model.cells[start[0]][start[1]].door:
            actionPointsCost += 1
            self.model.cells[start[0]][start[1]].door.remove(end)
            self.model.cells[end[0]][end[1]].door.remove(start)
            self.removeWall(end)
          if self.model.cells[end[0]][end[1]].fire == 2:
            actionPointsCost += 1
            self.model.firePoints.remove(self.model.cells[end[0]][end[1]])
            self.model.cells[end[0]][end[1]].fire = 0
            print(f"Fuego en {end} apagado en el camino por Agente {self.unique_id}")
          actionPointsCost = actionPointsCost + 1 * self.carryState
        elif start[0] > end[0]:
          if (self.model.cells[end[0]][end[1]].down or self.model.cells[start[0]][start[1]].up) and end not in self.model.cells[start[0]][start[1]].door:
            actionPointsCost += 4
            self.removeWall(end)
          elif (self.model.cells[end[0]][end[1]].down or self.model.cells[start[0]][start[1]].up) and end in self.model.cells[start[0]][start[1]].door:
            actionPointsCost += 1
            self.model.cells[start[0]][start[1]].door.remove(end)
            self.model.cells[end[0]][end[1]].door.remove(start)
            self.removeWall(end)
          if self.model.cells[end[0]][end[1]].fire == 2:
            actionPointsCost += 1
            self.model.firePoints.remove(self.model.cells[end[0]][end[1]])
            self.model.cells[end[0]][end[1]].fire = 0
            print(f"Fuego en {end} apagado en el camino por Agente {self.unique_id}")
          actionPointsCost = actionPointsCost + 1 * self.carryState
        elif start[1] < end[1]:
          if (self.model.cells[end[0]][end[1]].left or self.model.cells[start[0]][start[1]].right) and end not in self.model.cells[start[0]][start[1]].door:
            actionPointsCost += 4
            self.removeWall(end)
          elif (self.model.cells[end[0]][end[1]].left or self.model.cells[start[0]][start[1]].right) and end in self.model.cells[start[0]][start[1]].door:
            actionPointsCost += 1
            self.model.cells[start[0]][start[1]].door.remove(end)
            self.model.cells[end[0]][end[1]].door.remove(start)
            self.removeWall(end)
          if self.model.cells[end[0]][end[1]].fire == 2:
            actionPointsCost += 1
            self.model.firePoints.remove(self.model.cells[end[0]][end[1]])
            self.model.cells[end[0]][end[1]].fire = 0
            print(f"Fuego en {end} apagado en el camino por Agente {self.unique_id}")
          actionPointsCost = actionPointsCost + 1 * self.carryState
        elif start[1] > end[1]:
          if (self.model.cells[end[0]][end[1]].right or self.model.cells[start[0]][start[1]].left) and end not in self.model.cells[start[0]][start[1]].door:
            actionPointsCost += 4
            self.removeWall(end)
          elif (self.model.cells[end[0]][end[1]].right or self.model.cells[start[0]][start[1]].left) and end in self.model.cells[start[0]][start[1]].door:
            self.model.cells[start[0]][start[1]].door.remove(end)
            self.model.cells[end[0]][end[1]].door.remove(start)
            self.removeWall(end)
            actionPointsCost += 1
          if self.model.cells[end[0]][end[1]].fire == 2:
            actionPointsCost += 1
            self.model.firePoints.remove(self.model.cells[end[0]][end[1]])
            self.model.cells[end[0]][end[1]].fire = 0
            print(f"Fuego en {end} apagado en el camino por Agente {self.unique_id}")
          actionPointsCost = actionPointsCost + 1 * self.carryState
      return actionPointsCost


In [19]:
# @title
class MapModel(Model):
    def __init__(self, num_agents):
        super().__init__()
        self.savedLifes = 0
        self.width = 10
        self.height = 8
        self.structural_Damage_Left = 24
        self.num_agents = num_agents
        self.grid = MultiGrid(self.height, self.width, False)
        self.cells, self.outSide = self.read_map_data()
        self.putEntranceDoors()
        self.interestPoints = [cell for row in self.cells for cell in row if cell.alert != 0]
        self.firePoints = [cell for row in self.cells for cell in row if cell.fire == 2]
        self.schedule = RandomActivation(self)
        self.running = True
        for i in range(self.num_agents):
          random_pos = self.random.choice(self.outSide)
          while self.grid.is_cell_empty(random_pos.pos) == False:
            random_pos = self.random.choice(self.outSide)
          a = FiremanAgent(i, self)
          self.schedule.add(a)
          self.grid.place_agent(a, random_pos.pos)

    def read_map_data(self):
      with open('mapa.txt', 'r') as map:
        text = map.read()

        walls = []
        for i in range(8):
            for j in range(6):
                new_wall = text[:4]
                walls.append(new_wall)
                text = text[5:]

        alerts = []
        for i in range(3):
            pos_alert_x = text[0]
            pos_alert_y = text[2]
            pos_alert_state = text[4]
            text = text[6:]
            alerts.append( (pos_alert_x, pos_alert_y, pos_alert_state) )

        fires = []
        for i in range(10):
            pos_fire_x = text[0]
            pos_fire_y = text[2]
            text = text[4:]
            fires.append( (pos_fire_x, pos_fire_y) )

        doors = []
        for i in range(8):
            pos_doorA_x = text[0]
            pos_doorA_y = text[2]
            pos_doorB_x = text[4]
            pos_doorB_y = text[6]
            text = text[8:]
            doors.append( ( (pos_doorA_x, pos_doorA_y), (pos_doorB_x, pos_doorB_y) ) )

        exits = []
        for i in range(4):
            pos_exit_x = text[0]
            pos_exit_y = text[2]
            text = text[4:]
            exits.append( (pos_exit_x, pos_exit_y) )

        cells = []
        for i in range(6):
            for j in range(8):
                w = walls[0]
                del walls[0]

                c = Cell(i + 1,j + 1,w)
                cells.append(c)

                if (str(i + 1), str(j + 1), 'v') in alerts:
                    c.alert = 2
                elif (str(i + 1), str(j + 1), 'f') in alerts:
                    c.alert = 1

                if (str(i + 1), str(j + 1)) in fires:
                    c.fire = 2

                for d in doors:
                  if (str(i + 1), str(j + 1)) == d[0]:
                    c.door.append((int(d[1][0]), int(d[1][1])))
                  elif (str(i + 1), str(j + 1)) == d[1]:
                    c.door.append((int(d[0][0]), int(d[0][1])))

                if (str(i + 1), str(j + 1)) in exits:
                    c.entrance = True

        new_cells = [
            Cell(0, 0, "0000"),
            Cell(0, 1, "0010"),
            Cell(0, 2, "0010"),
            Cell(0, 3, "0010"),
            Cell(0, 4, "0010"),
            Cell(0, 5, "0010"),
            Cell(0, 6, "0010"),
            Cell(0, 7, "0010"),
            Cell(0, 8, "0010"),
            Cell(0, 9, "0000"),
        ]
        outSide = new_cells
        cells = new_cells + cells
        for i in range (1, 7):
          c = Cell(i, 0, "0001")
          cells.insert(i * 10, c)
          outSide.append(c)
          c = Cell(i, 9, "0100")
          cells.insert((i * 10) + 9, c)
          outSide.append(c)
        new_cells = [
            Cell(7, 0, "0000"),
            Cell(7, 1, "1000"),
            Cell(7, 2, "1000"),
            Cell(7, 3, "1000"),
            Cell(7, 4, "1000"),
            Cell(7, 5, "1000"),
            Cell(7, 6, "1000"),
            Cell(7, 7, "1000"),
            Cell(7, 8, "1000"),
            Cell(7, 9, "0000"),
        ]
        outSide = outSide + new_cells
        cells = cells + new_cells
        map = [[None for _ in range(10)] for _ in range(8)]
        for cell in cells:
          y, x = cell.pos
          map[y][x] = cell
          #print(f"{map[y][x].pos}: {map[y][x].up} - {map[y][x].left} - {map[y][x].down} - {map[y][x].right}   A: {map[y][x].alert}   F: {map[y][x].fire}   D: {map[y][x].door}    E: {map[y][x].entrance}")
        return map, outSide

    def step(self):
      self.asignPoints()
      for puntoInteres in self.interestPoints:
        print(f"Puntos de interés en: {puntoInteres.pos}")
      print("\n")
      for puntoFuego in self.firePoints:
        print(f"Puntos de fuego en: {puntoFuego.pos}")
      print("\n")
      for agent in model.schedule.agents:
        print(f"Agente: {agent.unique_id} Posición: {agent.pos} Yendo a: {agent.point.pos}")
      print("\n")
      self.schedule.step()

    def generateNewInterestPoint(self):
      flat_cells = [cell for row in self.cells for cell in row]
      randomCell = self.random.choice(list(filter(lambda cell: cell not in self.outSide and cell not in self.interestPoints, flat_cells)))
      randomCell.alert = self.random.randint(1, 2)
      return randomCell

    def putEntranceDoors(self):
      for row in self.cells:
        for cell in row:
          if cell.entrance:
            if cell.pos[0] == 1:
              cell.up = False
              self.cells[cell.pos[0] - 1][cell.pos[1]].down = False
            elif cell.pos[0] == 6:
              cell.down = False
              self.cells[cell.pos[0] + 1][cell.pos[1]].up = False
            elif cell.pos[1] == 1:
              cell.left = False
              self.cells[cell.pos[0]][cell.pos[1] - 1].right = False
            elif cell.pos[1] == 8:
              cell.right = False
              self.cells[cell.pos[0]][cell.pos[1] + 1].left = False

    def asignPoints(self):
      while len(self.interestPoints) < 3:
        self.interestPoints.append(self.generateNewInterestPoint())
      minSteps = 100
      closestAgent = None
      interestPoints = self.interestPoints.copy()
      for agent in self.schedule.agents:
        if agent.point in interestPoints:
          interestPoints.remove(agent.point)
      for interestPoint in interestPoints:
        closestAgent = None
        for agent in self.schedule.agents:
          if agent.point is None:
            steps = agent.dijkstra(agent.pos, interestPoint.pos)[1]
            if steps < minSteps:
              minSteps = steps
              closestAgent = agent.unique_id
        for agent in self.schedule.agents:
          if closestAgent == agent.unique_id:
            agent.point = interestPoint
            minSteps = 100
      minSteps = 100
      if len(self.firePoints) > 3:
        leftFirePoints = self.firePoints.copy()
        for agent in self.schedule.agents:
          if agent.point is None:
            for fire in leftFirePoints:
              steps = agent.dijkstra(agent.pos, fire.pos)[1]
              if steps < minSteps:
                minSteps = steps
                agent.point = fire
            leftFirePoints.remove(agent.point)
            minSteps = 100
      else:
        for agent in self.schedule.agents:
          if agent.point is None:
            agent.point = agent.point

In [22]:
model = MapModel(6)
for i in range(20):
  model.step()
print("Finalizado", model.savedLifes)
#for row in model.cells:
  #for cell in row:
    #print(f"{cell.pos}: {cell.up} - {cell.left} - {cell.down} - {cell.right}   A: {cell.alert}   F: {cell.fire}   D: {cell.door}    E: {cell.entrance}")

Puntos de interés en: (2, 4)
Puntos de interés en: (5, 1)
Puntos de interés en: (5, 8)


Puntos de fuego en: (2, 2)
Puntos de fuego en: (2, 3)
Puntos de fuego en: (3, 2)
Puntos de fuego en: (3, 3)
Puntos de fuego en: (3, 4)
Puntos de fuego en: (3, 5)
Puntos de fuego en: (4, 4)
Puntos de fuego en: (5, 6)
Puntos de fuego en: (5, 7)
Puntos de fuego en: (6, 6)


Agente: 0 Posición: (0, 1) Yendo a: (2, 4)
Agente: 1 Posición: (7, 6) Yendo a: (6, 6)
Agente: 2 Posición: (3, 9) Yendo a: (5, 8)
Agente: 3 Posición: (7, 3) Yendo a: (5, 1)
Agente: 4 Posición: (6, 0) Yendo a: (3, 2)
Agente: 5 Posición: (2, 9) Yendo a: (3, 5)


Falsa alarma en (5, 1)
Fuego en (6, 6) apagado en el camino por Agente 1
Persona encontrada en (5, 8)
Puntos de interés en: (2, 4)
Puntos de interés en: (2, 5)
Puntos de interés en: (6, 2)


Puntos de fuego en: (2, 2)
Puntos de fuego en: (2, 3)
Puntos de fuego en: (3, 2)
Puntos de fuego en: (3, 3)
Puntos de fuego en: (3, 4)
Puntos de fuego en: (3, 5)
Puntos de fuego en: (4, 4)